
**Install requirements**

In [1]:
!pip3 install torch torchvision torchaudio

In [2]:
!pip3 install 'tqdm'

**Import libraries**

In [3]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet

from PIL import Image
from tqdm import tqdm

from sklearn.model_selection import train_test_split

**Set Arguments**

In [4]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

NUM_CLASSES = 102 # 101 + 1: There is am extra Background class that should be removed

BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 1e-3            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 30      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

**Define Data Preprocessing**

In [5]:
# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

**Prepare Dataset**

Manteniamo traccia di ogni singola label e immagine che abbiamo visto nell'ordine in cui l'abbiamo vista.
Questo ci serve perché quando ci arriva una richiesta di get_item ad un indice specifico ci basterà accedere a queste due strutture dati all'indice indicato e prendere il rispettivo elemento di immagine e di label.

Memorizzaiamo un dizionario che ci serve per memorizzare la corretta associazione tra la label categorica ("cane") e il numero che indica la classe, per esempio 2.

andiamo a togliere 101_ObjectCategories e andiamo alla cartella che la contiene apriamo il file train o text e da li per ogni riga apro il path dato da 101_ObjectCategories riga Per risalire alla cartella "parent" che contiene root, puoi utilizzare la funzione os.path.dirname(root). Questa funzione restituisce il percorso della cartella padre (superiore) di quella specificata in root.

In [6]:
from torchvision.datasets import VisionDataset

from PIL import Image

import os
import os.path
import sys


def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')


class Caltech(VisionDataset):
    def __init__(self, root, split='train', transform=None, target_transform=None):
        super(Caltech, self).__init__(root, transform=transform, target_transform=target_transform)
        self.split = split

        self.images = []
        self.labels = []

        self.dim=0
        self.dizionario = dict()

        root = os.path.dirname(root)

        self.dizionario = dict()
        val_label=-1

        split_file = os.path.join(root, f"{split}.txt")
        with open(split_file, 'r') as f:
          for line in f:

              label, path = line.strip().split("/")

              if label!="BACKGROUND_Google":
                self.dim+=1

                self.images.append(os.path.join(root, '101_ObjectCategories', label, path))

                if label not in self.dizionario.keys():
                  val_label+=1
                  self.dizionario[label]=val_label
                self.labels.append(self.dizionario[label])
        '''
        - Labels should start from 0, so for Caltech you will have lables 0...100 (excluding the background class)
        '''

    def __getitem__(self, index):
        '''
        __getitem__ should access an element through its index
        Args:
            index (int): Index

        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        '''

        image, label = self.images[index], self.labels[index]
        image = pil_loader(self.images[index])
        # Applies preprocessing when accessing the image
        if self.transform is not None:
            image = self.transform(image)

        return image, self.labels[index]

    def __len__(self):
        '''
        The __len__ method returns the length of the dataset
        It is mandatory, as this is used by several other components
        '''
        length = len(self.images) # Provide a way to get the length (number of elements) of the dataset
        return self.dim
        #posso anche restituire length e mi esce lo stesso risultato

# Divisione training e validation set

In questa seziona andiamo ad effettuare la divisione tra training e validation set. L'idea è che ovviamente vogliamo fare in modo che il validation set rispetti il più possibile le diverse etichette dei dati. Ovvero dobbiamo fare in modo che il 25% dei dati di ogni classe venga preso nel validation set.

MA QUESTA COSA NON LA DEVO FARE MANUALE: MA CON TRAIN_TEST_SPIT E PASSANDO LO STRATIFY COME PARAMETRO.
CHE FA IN MODO CHE PRENDE IL 25% DI OGNI CLASSE.

In [7]:
# Clone github repository with data
import pathlib
print(pathlib.Path().resolve())
if not os.path.isdir('./Caltech101'):
  !git clone https://github.com/MachineLearning2020/Homework2-Caltech101.git
  !mv 'Homework2-Caltech101' 'Caltech101'

DATA_DIR = 'Caltech101/101_ObjectCategories'

# Prepare Pytorch train/test Datasets
train_dataset = Caltech(DATA_DIR, split='train',  transform=train_transform)
test_dataset = Caltech(DATA_DIR, split='test', transform=eval_transform)

#Dobbiamo splittare il dataset, per esempio 75% e 25%:
train_indexes = [] # split the indices for your train split
val_indexes = []# split the indices for your val split

num_train = len(train_dataset)
indices = list(range(num_train))

train_indexes, val_indexes = train_test_split(indices, test_size=0.25, random_state=42)

val_dataset = Subset(train_dataset, val_indexes)
train_dataset = Subset(train_dataset, train_indexes)

# Check dataset sizes
print('Train Dataset: {}'.format(len(train_dataset)))
print('Valid Dataset: {}'.format(len(val_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))

/content
Cloning into 'Homework2-Caltech101'...
remote: Enumerating objects: 9256, done.
remote: Total 9256 (delta 0), reused 0 (delta 0), pack-reused 9256 (from 1)
Receiving objects: 100% (9256/9256), 129.48 MiB | 13.70 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Updating files: 100% (9149/9149), done.
Train Dataset: 4338
Valid Dataset: 1446
Test Dataset: 2893


# Preparazione dei DataLoader

In [8]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


**Prepare Network**

Abbiamo già una funzione che crea una rete di tipo AlexNet. Tuttabia dobbiamo andare a modifcare il sesto livello di classificazione (fully connected) adattando il numero di neuroni risultato in base al numeor di classi che vogliamo classificare, in questo caso 101.

In [9]:
net = alexnet() # Loading AlexNet model

# AlexNet has 1000 output neurons, corresponding to the 1000 ImageNet's classes
# We need 101 outputs for Caltech-101
net.classifier[6] = nn.Linear(4096, NUM_CLASSES) # nn.Linear in pytorch is a fully connected layer
                                                 # The convolutional layer is nn.Conv2d

# We just changed the last layer of AlexNet with a new fully connected layer with 101 outputs
# It is strongly suggested to study torchvision.models.alexnet source code

**Prepare Training**

In [10]:
# Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Choose parameters to optimize
# To access a different set of parameters, you have to access submodules of AlexNet
# (nn.Module objects, like AlexNet, implement the Composite Pattern)
# e.g.: parameters of the fully connected layers: net.classifier.parameters()
# e.g.: parameters of the convolutional layers: look at alexnet's source code ;)
parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet

# Define optimizer
# An optimizer updates the weights based on loss
# We use SGD with momentum
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Define scheduler
# A scheduler dynamically changes learning rate
# The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
print(len(train_dataloader))

16


Nota che optimizer prende in modo esplicito come primo parametro i parametri che bisogna ottimizzare.

**Train**

In [11]:
# By default, everything is loaded to cpu
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime

current_step = 0

# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))

  # Iterate over the dataset
  for images, labels in train_dataloader:
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)

    # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1
    # Step the scheduler
    scheduler.step()



Starting epoch 1/30, LR = [0.001]


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:536: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  _warn_get_lr_called_within_step(self)


Step 0, Loss 4.623244762420654
Step 10, Loss 4.621798038482666
Starting epoch 2/30, LR = [0.001]
Step 20, Loss 4.619715690612793
Step 30, Loss 4.621179103851318
Starting epoch 3/30, LR = [0.0001]
Step 40, Loss 4.618213653564453
Starting epoch 4/30, LR = [1e-05]
Step 50, Loss 4.619962215423584
Step 60, Loss 4.618201732635498
Starting epoch 5/30, LR = [1.0000000000000002e-06]
Step 70, Loss 4.618915557861328
Starting epoch 6/30, LR = [1.0000000000000004e-08]
Step 80, Loss 4.619654655456543
Step 90, Loss 4.619086742401123
Starting epoch 7/30, LR = [1.0000000000000002e-07]
Step 100, Loss 4.619754791259766
Step 110, Loss 4.619565010070801
Starting epoch 8/30, LR = [1.0000000000000004e-08]
Step 120, Loss 4.619837284088135
Starting epoch 9/30, LR = [1.0000000000000005e-09]
Step 130, Loss 4.617499828338623
Step 140, Loss 4.6189751625061035
Starting epoch 10/30, LR = [1.0000000000000006e-10]
Step 150, Loss 4.619433403015137
Starting epoch 11/30, LR = [1.0000000000000006e-12]
Step 160, Loss 4.618

KeyboardInterrupt: 

# best parameters
Andiamo ad usare il validation set per trovare la migliore combinazione di parametri (non iperparametri) della rete.

Dopo ogni epoca andimo a vedere i risultti che otteniano per quella epoca, magari considerando l'accuracy e vedo come quella rete (con i parametri attuali) performano sui dati di validation.

Tramite il metodo net.state_dict() vado a prendere tutti i parametri della rete.

In [ ]:
best_loss = float('inf')
best_net = None

net = net.to(DEVICE)
cudnn.benchmark
current_step = 0

for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))

  for images, labels in train_dataloader:
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)
    net.train()
    optimizer.zero_grad()
    outputs = net(images)
    loss = criterion(outputs, labels)
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))
    loss.backward()
    optimizer.step()
    current_step += 1
    scheduler.step()

  #verifico la rete sui dati di validazione:
  with torch.no_grad():
    for images, labels in val_dataloader:
          images = images.to(DEVICE)
          labels = labels.to(DEVICE)
          net.train()
          output=net(images)
          loss += criterion(output, labels)

  loss = loss/len(val_dataloader)
  if loss<best_loss:
      best_loss=loss
      best_net = net.state_dict()

#net.load_state_dict(best_net)

# Validation
Adesso andiamo ad effettuare il processo di validation, ovvero andiamo a scegliere opportuna i valori degli IPER PARAMETRI DELLA RETE.

Per la traccia, dobbiamo effettuare validation almeno per due iperparametri. Di seguito si mostra la validazione per gli iperparametri di lr e epoche.

Per effettuare la validazione possiamo considerare una lista di possibili valori per ognuno di questi due iperparametri e poi vedere i risultati per ongi combinazione di questi iperparametri.

In [ ]:
from itertools import product

net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

#Costruisco la lista degli iper parametri che voglio testare:
lr = [0.1, 0.01, 0.001, 0.0001]
NUM_EPOCHS = [5, 10, 15, 30]
best_accuracy = 0
best_lr=None
best_epochs=None
combinations = list(product(lr, NUM_EPOCHS))

# Stampare le combinazioni per verificare
for lr_val, epoch_val in combinations:
  criterion = nn.CrossEntropyLoss()
  parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet
  optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
  scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

  #adesso vediamo come si comporta la rete neurale con questi parametri:
  net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

  cudnn.benchmark # Calling this optimizes runtime

  current_step = 0
  running_corrects = 0
    # NOTA CHE DOBBIAMO USARE IL DATASET DI VALIDATION
  for images, labels in val_dataloader:
      # Bring data over the device of choice
      images = images.to(DEVICE)
      labels = labels.to(DEVICE)
      # Forward Pass
      outputs = net(images)
      # Get predictions
      _, preds = torch.max(outputs.data, 1)
      # Update Corrects
      running_corrects += torch.sum(preds == labels.data).data.item()
    # Calculate Accuracy
  accuracy = running_corrects / float(len(val_dataset))
  if accuracy>best_accuracy:
    best_accuracy = accuracy
    best_lr = lr_val
    best_epochs = epoch_val


print("the best accuracy is: ", best_accuracy)
print("the best parameters of lr, num_epochs are: ", best_lr, ", ", best_epochs)

**Test**

In [ ]:
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))

# Transfer learning

Andiamo a congelare i parametri della rete Alexnet trainata su imagenet dataset e modifichiamo l'ultimo livello per considerare il numeor di classi di nostro interesse e addestriamo solo l'ultimo livello fully convolutional.

In [ ]:
import torchvision.models as models

alexnet = models.alexnet(pretrained=True)

In questo modo stiamo andando a caricare il modello AlexNet con i pesi di ImageNet.

Sostituisco l'ultimo livello: nella rete AlexNet erano presenti 1000 classi, a noi ne bastano 102:

In [ ]:
num_classes = 102
alexnet.classifier[6] = nn.Linear(4096, num_classes)

 Adesso dobbiamo indicare la volontà di **congelare alcuni parametri** della rete. Per farlo possiamo dichiarare quali parametri della rete non richiedono il calcolo del gradiente.

In [ ]:
for param in alexnet.parameters():
  param.requires_grad = False

In questo modo abbiamo congelato tutti i parametri della rete, per indicare invece che alcuni di questi non debbano essere congelati possiamo fare:

In [ ]:
for param in alexnet.classifier[6].parameters():
  param.requires_grad = True

Adesso dobbiamo configurare l'**ottimizzatore** per indicare che deve addestrare solo i pesi dell'ultimo livello.

In [ ]:
optimizer = optim.SGD(alexnet.classifier[6].parameters(), lr=0.001, momentum=0.9)

Training della rete.

In [ ]:
# By default, everything is loaded to cpu
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime

current_step = 0
NUM_EPOCHS = 50 #Adesso il training dovrebbe metterci di meno

# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))

  # Iterate over the dataset
  for images, labels in train_dataloader:
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)

    # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1
    # Step the scheduler
    scheduler.step()



Test:

In [ ]:
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))

# train only the convolutional layers

Chiaramente non ha molto senso.

In [ ]:
import torchvision.models as models
alexnet = models.alexnet(pretrained=True)
num_classes = 102
alexnet.classifier[6] = nn.Linear(4096, num_classes)
for param in alexnet.classifier.parameters():
  param.requires_grad = False


In questo modo congeliamo tutti i livelli, tranne quelli convoluzionali.

Nota che features (alexnet) indica i livelli convoluzionali e classifier quelli FC.

In [ ]:
optimizer = optim.SGD(alexnet.features.parameters(), lr=0.001, momentum=0.9)

Train

In [ ]:
# By default, everything is loaded to cpu
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime

current_step = 0
NUM_EPOCHS = 50 #Adesso il training dovrebbe metterci di meno

# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))

  # Iterate over the dataset
  for images, labels in train_dataloader:
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)

    # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1
    # Step the scheduler
    scheduler.step()

Test

In [ ]:
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))

# Data augmentation

Andiamo ad effettuare delle operazioni di capovolgimento dell'immagine o operazioni sui colori, combiniamoli assieme e otteniamo delle nuove immagini trasformate. Aggiungiamo queste immagini al dataset ed effettuiamo il training su un dataset "aumentato".

In [ ]:
import torch
from torchvision.transforms import v2

color_jitter = transforms.ColorJitter(
    brightness=0.5,  # Fattore di luminosità (0 è nero, 1 è invariato, valori >1 aumentano la luminosità)
    contrast=0.5,    # Fattore di contrasto (0 è un'immagine grigia uniforme, 1 è invariato)
    saturation=0.5,  # Fattore di saturazione (0 è in bianco e nero, 1 è invariato)
    hue=0.1          # Spostamento della tonalità (da -0.5 a 0.5)
)

augmentation_transform =  transforms.Compose([
    color_jitter,
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
])

Ricorda che la dimensione effettiva delle immagini deve essere la stessa, per evitare problemi considero le operazioni di data aumentation aggiunte alle canoniche operazioni di trasformazione.

In [ ]:
# Clone github repository with data
from torch.utils.data import ConcatDataset
import pathlib
print(pathlib.Path().resolve())
if not os.path.isdir('./Caltech101'):
  !git clone https://github.com/MachineLearning2020/Homework2-Caltech101.git
  !mv 'Homework2-Caltech101' 'Caltech101'

DATA_DIR = 'Caltech101/101_ObjectCategories'

# Prepare Pytorch train/test Datasets
train_dataset_original = Caltech(DATA_DIR, split='train',  transform=train_transform)
train_dataset_augmented = Caltech(DATA_DIR, split='train',  transform=augmentation_transform)
train_dataset = ConcatDataset([train_dataset_original, train_dataset_augmented])

test_dataset = Caltech(DATA_DIR, split='test', transform=eval_transform)

#Dobbiamo splittare il dataset, per esempio 75% e 25%:
train_indexes = [] # split the indices for your train split
val_indexes = []# split the indices for your val split

num_train = len(train_dataset)
indices = list(range(num_train))

train_indexes, val_indexes = train_test_split(indices, test_size=0.25, random_state=42)

val_dataset = Subset(train_dataset, val_indexes)
train_dataset = Subset(train_dataset, train_indexes)

# Check dataset sizes
print('Train Dataset: {}'.format(len(train_dataset)))
print('Valid Dataset: {}'.format(len(val_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))

In questo modo:

train_dataset_original = Caltech(DATA_DIR, split='train',  transform=train_transform)


train_dataset_augmented = Caltech(DATA_DIR, split='train',  transform=augmentation_transform)


train_dataset = ConcatDataset([train_dataset_original, train_dataset_augmented])


Stiamo creando due dataset: quello originale e quello aumentato e poi li concateniamo.

In [ ]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [ ]:
#net = alexnet()
net.classifier[6] = nn.Linear(4096, NUM_CLASSES)

In [ ]:
# Define loss function
criterion = nn.CrossEntropyLoss()
parameters_to_optimize = net.parameters()
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
print(len(train_dataloader))


In [ ]:

# By default, everything is loaded to cpu
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime

current_step = 0

# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))

  # Iterate over the dataset
  for images, labels in train_dataloader:
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)

    # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1
    # Step the scheduler
    scheduler.step()

In [ ]:
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))